In [ ]:
pip install -q kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!touch ~/.kaggle/kaggle.json

In [ ]:
api_token = {"username":"", "key":""} #get these from your kaggle account

In [ ]:
import json

In [ ]:
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

In [ ]:
chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia --force

In [ ]:
import zipfile
with zipfile.ZipFile('./chest-xray-pneumonia.zip', 'r') as zip_ref:
    zip_ref.extractall('./data')

In [ ]:
import glob
import random
import matplotlib.pyplot as plt

def get_random_image (dir, condition):
    placeholder=''
    if condition=='n':
        placeholder='NORMAL'
    elif condition=='p':
        placeholder='PNEUMONIA'

    else:
        raise Exception ("Sorry Invalid Condition")
    folder = f"./data/chest_xray/{dir}/{placeholder}/*.jpeg"
    img_paths = glob.glob(folder)
    max_length = len(img_paths)
    randomNumber = random.randint(0, max_length)
    
    for index,item in enumerate(img_paths, start=1):
        if index==randomNumber:
            print(index,item)
            image=plt.imread(item)
            readyImage=plt.imshow(image)
            return readyImage

In [ ]:
get_random_image("train", "p")
# get_random_image("val", "p")
# get_random_image("val", "n")

We will now be extracting images to their correct folder structure AND resizing the images.
train_pneumonia0.jpeg, train_normal0.jpeg
In Sagemaker we will be doing transfer learning using the Image classifier which relies on the RESNET framework
The images have to be 224 by 224 and 3 colour channels - RGB
When using matplotlib, greyscale and B&W images are savaed and opened withe the default colour map - vidris
We need to save images in grayscale the actual colour code of x-rays

In [ ]:
# We have the mode L currently - 2 colour channels
# When we specify the configurations for our training job in Sagemaker, we will specify that inputs should be 224 by 224 by 3

from PIL import Image
image=Image.open('path to chest x-ray image')
print(image.format)
print(image.size)
print(image.mode)

In [ ]:
# PNGs have RGBA - 4 colour channels, when this happens change it before starting the training job
import PIL.Image
rgba_image = PIL.Image.open('path to rgba image')
rgb_image = rgba_image.convert('RGB')
print(rgb_image.mode)

In [ ]:
# When you open the pneumonia folder you will see every image label starts with "person"
# When you open the normal folder you will see every image label starts with "IM"
import glob
import matplotlib.pyplot as plt
from PIL import Image


folder = '.data/chest_xray/train/*/*.jpeg'

counterPneu=0
counterNormal=0

img_paths = glob.glob(folder)

for i in img_paths:
    if "person" in i:
        full_size_image = Image.open(i)
        im = full_size_image.resize((224, 224))
        plt.imsave(fname='./data/chest_xray/train' + '/train_pneumonia' + str(counterPneu) + '.jpeg', arr=im, format='jpeg', cmap='gray')
        counterPneu+=1
    else:
        full_size_image = Image.open(i)
        im = full_size_image.resize((224, 224))
        plt.imsave(fname='./data/chest_xray/train' + '/train_normal' + str(counterNormal) + '.jpeg', arr=im, format='jpeg', cmap='gray')
        counterNormal+=1

In [ ]:
import glob
import matplotlib.pyplot as plt
from PIL import Image

folder = '.data/chest_xray/test/*/*.jpeg'

counterPneu=0
counterNormal=0

img_paths = glob.glob(folder)

for i in img_paths:
    if "person" in i:
        full_size_image = Image.open(i)
        im = full_size_image.resize((224, 224))
        plt.imsave(fname='./data/chest_xray/test' + '/test_pneumonia' + str(counterPneu) + '.jpeg', arr=im, format='jpeg', cmap='gray')
        counterPneu+=1
    else:
        full_size_image = Image.open(i)
        im = full_size_image.resize((224, 224))
        plt.imsave(fname='./data/chest_xray/test' + '/test_normal' + str(counterNormal) + '.jpeg', arr=im, format='jpeg', cmap='gray')
        counterNormal+=1

In [ ]:
# import glob
# import matplotlib.pyplot as plt
# from PIL import Image

folder = '.data/chest_xray/val/*/*.jpeg'

counterPneu=0
counterNormal=0

img_paths = glob.glob(folder)

for i in img_paths:
    if "person" in i:
        full_size_image = Image.open(i)
        im = full_size_image.resize((224, 224))
        plt.imsave(fname='./data/chest_xray/val' + '/val_pneumonia' + str(counterPneu) + '.jpeg', arr=im, format='jpeg', cmap='gray')
        counterPneu+=1
    else:
        full_size_image = Image.open(i)
        im = full_size_image.resize((224, 224))
        plt.imsave(fname='./data/chest_xray/val' + '/val_normal' + str(counterNormal) + '.jpeg', arr=im, format='jpeg', cmap='gray')
        counterNormal+=1

AFTER RESIZING AND MAKING GRAY, WE CAN NOW DELETE THE NORMAL AND PNEUMONIA FOLDERS FROM THE TEST,TRAIN AND VAL FOLDERS.

In [ ]:
# We are creating a dataframe so that we can visualize on seaborn:
# 3 columns - folder, lung condition, file path
# row example: val, normal, filepath.jpeg
# row example: test, pneumonia, filepath.jpeg

import glob
import pandas as pd

folder = './data/chest_xray/*/*.jpeg'

category = []
condition_of_lung = []
filenames = []

all_files = glob.glob(folder)

for filename in all_files: 
    if "train" in filename:
        if "pneumonia" in filename:
            category.append("train")
            filenames.append(filename)
            condition_of_lung.append("pneumonia")
        elif "normal" in filename:
            category.append("train")
            filenames.append(filename)
            condition_of_lung.append("normal")
    elif "test" in filename:
        if "pneumonia" in filename:
            category.append("test")
            filenames.append(filename)
            condition_of_lung.append("pneumonia")
        elif "normal" in filename:
            category.append("test")
            filenames.append(filename)
            condition_of_lung.append("normal")
    elif "val" in filename:
        if "pneumonia" in filename:
            category.append("val")
            filenames.append(filename)
            condition_of_lung.append("pneumonia")
        elif "normal" in filename:
            category.append("val")
            filenames.append(filename)
            condition_of_lung.append("normal")

all_data_df = pd.DataFrame ({"dataset type":category, "x-ray reult":condition_of_lung, "filename":filenames})
print(all_data_df.head())

In [ ]:
import seaborn as sns
g = sns.catplot(x="x-ray result", col="dataset type", kind="count", palette="ch:.55", data= all_data_df, legend=True)

for i in range (0.3):
    ax = g.facet_axis(0,i)
    for p in ax.patches:
        ax.text(p.get_x()+0.3,
                p.get_height()*1.05,
                '{0:0f}'.format(p.get_height()),
                color = 'black',
                rotation='horizontal',
                size='large')  

In [ ]:
# Creating lst filess

import glob
import pandas as pd
import os
train_folder = './data/chest_xray/train/*.jpeg'
train_df_lst = pd.DataFrame(columns=['labels', 's3_path'], dtype=object)
train_imgs_path = glob.glob(train_folder)
counter = 0
class_arg = ''

for i in train_imgs_path:
    if "pneumonia" in i:    
        class_arg = 1
    else:
        class_arg = 0
    train_df_lst.loc[counter] = [class_arg, os.path.basename(i)]
    counter+=1
print(train_df_lst.head())

In [ ]:
# Creating lst filess

import glob
import pandas as pd
import os
test_folder = './data/chest_xray/test/*.jpeg'
test_df_lst = pd.DataFrame(columns=['labels', 's3_path'],dtype=object)
test_imgs_path = glob.glob(test_folder)
counter = 0
class_arg = ''

for i in test_imgs_path:
    if "pneumonia" in i:
        class_arg = 1
    else:
        class_arg = 0
    test_df_lst.loc[counter] = [class_arg, os.path.basename(i)]
    counter+=1
print(test_df_lst.head())

In [ ]:
def save_to_lst(df,prefix):
    return df [["labels", "s3_path"]].to_csv(
        f"{prefix}.lst", sep='\t', index=True, header=False
    )

save_to_lst(train_df_lst.copy(),"train")
save_to_lst(test_df_lst.copy, "test")

In [ ]:
# Upload to S3
bucket = 's3_bucket_name' # your bucket name
print("bucket:{}".format(bucket))
region = 'eu-central-1' # or the region you are working on
print("region:{}".format(region))
roleArn = 'arn:aws:s3:::s3_bucket_name' #your bucket arn
print("roleArn:{}".format(roleArn))

In [ ]:
import os
os.environ["DEFAULT_S3_BUCKET"] = bucket

In [ ]:
!aws s3 sync .data/chest_xray/train s3://${DEFAULT_S3_BUCKET}/train/

In [ ]:
!aws s3 sync .data/chest_xray/test s3://${DEFAULT_S3_BUCKET}/test/

In [ ]:
import boto3

boto3.Session().resource('s3').Bucket(bucket).Object("train.lst").upload_file('./train.lst')


In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object("test.lst").upload_file('./test.lst')  

In [ ]:
import sagemaker
from sagemaker import image_uris
import boto3
from sagemaker import get_execution_role
sess=sagemaker.Session()

algorithm_image=image_uris.retrieve(
    region = boto3.Session().region_name,
    framework="image-classification"
)

s3_output_location = f"s3://{bucket}/models/image_model"
print(algorithm_image)

In [ ]:
role = get_execution_role()
print(role)

In [ ]:
img_classifier_model = sagemaker.estimator.Estimator(
    algorithm_image, 
    role=role,
    instance_count=1,
    instance_type="ml.p2.xlarge",
    volume_size=50,
    max_run=432000
    input_mode="File", # for pipe mode you need protobuf format, best with millions of data records
    output_path=s3_output_location,
    sagemaker_session = sess
)

print (img_classifier_model)

# Hyperparameter tuning

In [ ]:
import glob
count = 0

for filepath in glob.glob('./data/chest_xray/train/*.jpeg'):
    count+=1
print (count)


In [ ]:
img_classifier_model.set_hyperparameters(
    image_shape="3,224,224", # sagemaker will transform the gray scale to RGB due to the specification made
    num_classes=2,
    use_pretrained_model=1,
    num_training_samples=count,
    augmentation_type='crop_color_transformation',
    epochs=15, # for enterprise bare minimum is 50 epochs to ensure high accuracy
    early_stopping=True,
    early_stopping_min_epochs=8, # if epochs were 50, this would be 30
    early_stopping_patience=5,
    early_stopping_tolerance=0.0, # if (improvement in accuracy / previous best accuracy) is less than early stopping tolerance, then there is no improvement
    lr_scheduler_factor=0.1,
    lr_scheduler_step='8, 10, 12' #if epochs were 50, '25, 30, 35'
)

In [ ]:
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner
hyperparameter_ranges={
    "learning_rate":ContinuousParameter(0.01,0.1),
    "mini_batch_size":CategoricalParameter([8,16,32]), # if we had a larger dataset ([256,512,1024])
    "optimizer":CategoricalParameter(["sgd", "adam"])
}

In [ ]:
objective_metric_name="validation:accuracy"
objective_type="Maximize"
max_jobs=5 # we have a small dataset
max_parallel_jobs=1

In [ ]:
tuner = HyperparameterTuner(estimator=img_classifier_model,
                            objective_metric_name=objective_metric_name,
                            hyperparameter_ranges=hyperparameter_ranges,
                            objective_type=objective_type,
                            max_jobs=max_jobs,
                            max_parallel_jobs=max_parallel_jobs
                            )

In [ ]:
from sagemaker.session import TrainingInput

model_inputs={
    "train":sagemaker.inputs.TrainingInput(s3_data=f"s3://{bucket}/train/", content_type="application/x-image"),
    "validation":sagemaker.inputs.TrainingInput(s3_data=f"s3://{bucket}/test/", content_type="application/x-image"),
    "train_lst":sagemaker.inputs.TrainingInput(s3_data=f"s3://{bucket}/train.lst/", content_type="application/x-image"),
    "validation_lst":sagemaker.inputs.TrainingInput(s3_data=f"s3://{bucket}/test.lst/", content_type="application/x-image"),
}

In [ ]:
import time
job_name_prefix="classifier"
timestamp=time.strftime("-%Y-%m-%d-%H-%M-%S", time.gmtime())
job_name=job_name_prefix+timestamp

In [ ]:
tuner.fit(inputs=model_inputs,job_name=job_name,logs=True)

In [ ]:
import sagemaker
from sagemaker import get_execution_role

In [ ]:
role = get_execution_role()

In [ ]:
model=sagemaker.model.Model(
    image_uri = algorithm_image, #re-run algorithm image cell,
    model_data = 'get/the/s3 URI of the best model',
    role=role
)

In [ ]:
endpoint_name='Demo-image-classifier-Pneumonia'

deployment = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    endpoint_name=endpoint_name
)

In [ ]:
# Once endpoint is deployed, run local inference
from sagemaker.predictor import Predictor
predictor = Predictor("Demo-image-classifier-Pneumonia")

In [ ]:
from sagemaker.serializers import IdentitySerializer
import base64

file_name = 'data/chest_xray/val/val_normal0.jpeg'
predictor.serializer=IdentitySerializer("image/jpeg")
with open (file_name, "rb") as f:
    payload = f.read()

inference=predictor.predict(data=payload)
print(inference)

In [ ]:
# Confusion Matrix, precision, recall, deserialization

import glob
import json
import numpy as np
file_path = 'data/chest_xray/val/*.jpeg'
files=glob.glob(file_path)

y_true=[]
y_pred=[]

def make_pred():
    for file in files:
        if "normal" in file:
            with open (file, 'rb') as f:
                payload = f.read()
                inference = predictor.predicst(data=payload).decode("utf-8")
                result = json.loads(inference)
                predicted_class = np.argmax(result)
                y_true.append(0)
                y_pred.append(predicted_class)
        elif "pneumonia" in file:
            with open (file, 'rb') as f:
                payload = f.read()
                inference = predictor.predict(data=payload).decode("utf-8")
                result = json.loads(inference)
                predicted_class = np.argmax(result)
                y_true.append(1)
                y_pred.append(predicted_class)

make_pred()
print(y_true)
print(y_pred)

In [ ]:
# Basic Confusion Matrix 
from sklearn.metrics import confusion_matrix

confusion_matrix(y_true, y_pred)

In [ ]:
# classification report

from sklearn.metrics import classification_report
print (classification_report(y_true, y_pred))